# Conexión y extracción de datos de Spotipy 

In [1]:
pip install spotipy

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Importamos todo lo necesario para conectarnos a la API y trabajar con los datos
import pandas as pd
import requests
import re
import json
import base64

pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames
pd.set_option('display.max_rows', None)  # Muestra todas las filas


# Librería Spotify
import spotipy

from spotipy.oauth2 import SpotifyClientCredentials



In [2]:
# Nuestras credenciales.

CLIENT_ID = 'db2acab1f0c14af9ba518442a04903c7'
CLIENT_SECRET = '1c3ced693d67453ea6b3ecf4d0df6008'

# Autenticación
auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [3]:
#Estos son los metodos de busqueda en la API
datos = sp.search(q="genre:rock year:2019", type='track', limit=50, offset=0)
data_track_album = sp.search(q="genre:rock year:2019", type='track,album', limit=50, offset=0)#para ambos

In [5]:
data_track_album.keys()

dict_keys(['tracks', 'albums'])

In [6]:
datos.keys()

dict_keys(['tracks'])

In [4]:
#Variables para la consulta.

genres = ['pop', 'rock', 'indie', 'reggae']
anio_inicio = 2019
anio_final = 2025

results = []
artistas = []

In [5]:
# Definimos el bucle para la selección de datos y también el número de registros que tenemos en este caso son 300 (ofset, indica cada vez que empieza una pagina nueva). 

offsets = [0, 50, 100, 150, 200 , 250]

for genre in genres:
    for offset in offsets:
        results_genre = sp.search(
            q=f'genre:{genre} year:{anio_inicio}-{anio_final}',
            type='track,album',
            limit=50,
            offset=offset
        )

        # Procesar canciones
        for item in results_genre.get('tracks', {}).get('items', []):
            artistas.append(item['artists'][0]['name'])
            results.append({
                'name_track': item['name'],
                'name_artist': item['artists'][0]['name'],
                'genre': genre,
                'type': item['type'],
                'year': item['album']['release_date'][:4]
            })

        # Procesar álbumes (solo en el último offset si quieres como antes)
        if offset == 250:
            for item in results_genre.get('albums', {}).get('items', []):
                artistas.append(item['artists'][0]['name'])
                results.append({
                    'name_track': item['name'],
                    'name_artist': item['artists'][0]['name'],
                    'genre': genre,
                    'type': item['type'],
                    'year': item['release_date'][:4]
                })

print(len(results))

1200


In [11]:
list(results)

[{'name_track': 'El Efecto - Remix',
  'name_artist': 'Rauw Alejandro',
  'genre': 'pop',
  'type': 'track',
  'year': '2019'},
 {'name_track': 'LUNA',
  'name_artist': 'Feid',
  'genre': 'pop',
  'type': 'track',
  'year': '2023'},
 {'name_track': 'Vagabundo',
  'name_artist': 'Sebastian Yatra',
  'genre': 'pop',
  'type': 'track',
  'year': '2023'},
 {'name_track': 'Algo Sencillito',
  'name_artist': 'Marta Santos',
  'genre': 'pop',
  'type': 'track',
  'year': '2023'},
 {'name_track': 'El Fin del Mundo',
  'name_artist': 'La La Love You',
  'genre': 'pop',
  'type': 'track',
  'year': '2020'},
 {'name_track': 'Die With A Smile',
  'name_artist': 'Lady Gaga',
  'genre': 'pop',
  'type': 'track',
  'year': '2024'},
 {'name_track': 'Me has invitado a bailar',
  'name_artist': 'Dani Fernández',
  'genre': 'pop',
  'type': 'track',
  'year': '2024'},
 {'name_track': 'Me has invitado a bailar',
  'name_artist': 'Dani Fernández',
  'genre': 'pop',
  'type': 'track',
  'year': '2024'},
 {'

In [6]:
# Establecemos el DataFrame con los resultados obtenidos y lo mostramos.
df_results = pd.DataFrame(results)

df_results

,name_track,name_artist,genre,type,year
0,El Efecto - Remix,Rauw Alejandro,pop,track,2019
1,LUNA,Feid,pop,track,2023
2,Vagabundo,Sebastian Yatra,pop,track,2023
3,Algo Sencillito,Marta Santos,pop,track,2023
4,El Fin del Mundo,La La Love You,pop,track,2020
5,Die With A Smile,Lady Gaga,pop,track,2024
6,Me has invitado a bailar,Dani Fernández,pop,track,2024
7,Me has invitado a bailar,Dani Fernández,pop,track,2024
8,BIRDS OF A FEATHER,Billie Eilish,pop,track,2024
9,Soltera,Shakira,pop,track,2024


In [7]:
# Guardamos el DataFrame en un archivo CSV
df_results.to_csv('datos_spotify_3.csv', index=False, encoding='utf-8')

In [11]:
# Cargar el archivo CSV
df = pd.read_csv('datos_spotify_3.csv')



In [12]:
# Eliminar duplicados
df_sin_duplicados = df.drop_duplicates()

# Guardar el nuevo archivo sin duplicados
df_sin_duplicados.to_csv('archivo_sin_duplicados.csv', index=False)

df_sin_duplicados

,name_track,name_artist,genre,type,year
0,El Efecto - Remix,Rauw Alejandro,pop,track,2019
1,LUNA,Feid,pop,track,2023
2,Vagabundo,Sebastian Yatra,pop,track,2023
3,Algo Sencillito,Marta Santos,pop,track,2023
4,El Fin del Mundo,La La Love You,pop,track,2020
5,Die With A Smile,Lady Gaga,pop,track,2024
6,Me has invitado a bailar,Dani Fernández,pop,track,2024
8,BIRDS OF A FEATHER,Billie Eilish,pop,track,2024
9,Soltera,Shakira,pop,track,2024
10,El Principio de algo,La La Love You,pop,track,2023


In [13]:
# Obtener la lista de artistas únicos.

unique_artists = pd.Series(artistas).unique()
unique_artists

array(['Rauw Alejandro', 'Feid', 'Sebastian Yatra', 'Marta Santos',
       'La La Love You', 'Lady Gaga', 'Dani Fernández', 'Billie Eilish',
       'Shakira', 'Benson Boone', 'Bizarrap', 'Leiva', 'Manuel Turizo',
       'Ozuna', 'Lola Indigo', 'Manuel Carrasco', 'Marc Seguí',
       'Arde Bogotá', 'Viva Suecia', 'zzoilo', 'DePol', 'Lola Young',
       'Chappell Roan', 'Miley Cyrus', 'Danny Ocean', 'ROSALÍA', 'Tainy',
       'Maluma', 'Harry Styles', 'David Guetta', 'Emilia', 'The Weeknd',
       'Sabrina Carpenter', 'Lewis Capaldi', 'Boza', 'Myles Smith',
       'Gigi Perez', 'Dimitri Vegas & Like Mike', 'Ovy On The Drums',
       'Camilo', 'd4vd', 'Tate McRae', 'Dua Lipa', 'Olivia Rodrigo',
       'Taylor Swift', 'Djo', 'Hozier', 'Mark Ambor', 'Luis Fonsi',
       'Maria Becerra', 'Ofenbach', 'Jimin', 'Ed Sheeran',
       'Ariana Grande', 'Mau y Ricky', 'The Marías', 'Hoobastank',
       'Adele', 'Måneskin', 'Post Malone', 'James Arthur', 'Kygo',
       'Glass Animals', 'Zhamira Zambr

In [ ]:
#Eliminar si no es necesario.

# Transformar la lista de artistas únicos en un DataFrame y guardarlo en un archivo CSV.
df_artistas = pd.DataFrame(unique_artists)
df_artistas.to_csv('datos_artistas_3.csv', index=False, encoding='utf-8')

Lista_artistas_unicos= df['Nombre_artista'].unique()
df_artistas=pd.DataFrame(Lista_artistas_unicos)


# Cargamos los datos a SQL

In [13]:
# Importar las librerías necesarias para la conexión a la base de datos MySQL

import mysql.connector
from mysql.connector import errorcode
import pandas as pd

from sqlalchemy import create_engine,FLOAT, VARCHAR, INTEGER, DATE, SmallInteger
from sqlalchemy.sql.sqltypes import String

In [19]:
#Buscar la ruta y mostrar el archivo CSV.

df_sin_duplicados = pd.read_csv(r'/Users/genesisnarea/Desktop/Escritorio - MacBook Air de Genesis/ADALAB/Modulo 2/proyecto_modulo_2/Definitivos/CSV datos_spotify_3_sin_duplicados.csv')

df_sin_duplicados 

,Nombre_artista,Tipo,Nombre_cancion,Genero,Anio
0,Rauw Alejandro,track,El Efecto - Remix,pop,2019
1,Feid,track,LUNA,pop,2023
2,Sebastian Yatra,track,Vagabundo,pop,2023
3,Marta Santos,track,Algo Sencillito,pop,2023
4,La La Love You,track,El Fin del Mundo,pop,2020
5,Lady Gaga,track,Die With A Smile,pop,2024
6,Dani Fernández,track,Me has invitado a bailar,pop,2024
7,Billie Eilish,track,BIRDS OF A FEATHER,pop,2024
8,Shakira,track,Soltera,pop,2024
9,Benson Boone,track,Beautiful Things,pop,2024


In [20]:
print(df_sin_duplicados.columns)

Index(['Nombre_artista', 'Tipo', 'Nombre_cancion', 'Genero', 'Anio'], dtype='object')


In [ ]:
#NO EJECUTAR !!!!!  Cambiar el nombre de las columnas para que sean iguales que SQL. 

df = df_sin_duplicados.rename(columns={
    'name_artist': 'Nombre_artista',
    'type': 'Tipo',
    'name_track': 'Nombre_cancion',
    'genre': 'Genero',
    'year': 'Anio'
})


df

,Nombre_cancion,Nombre_artista,Genero,Tipo,Anio
0,El Efecto - Remix,Rauw Alejandro,pop,track,2019
1,LUNA,Feid,pop,track,2023
2,Vagabundo,Sebastian Yatra,pop,track,2023
3,Algo Sencillito,Marta Santos,pop,track,2023
4,El Fin del Mundo,La La Love You,pop,track,2020
5,Die With A Smile,Lady Gaga,pop,track,2024
6,Me has invitado a bailar,Dani Fernández,pop,track,2024
7,BIRDS OF A FEATHER,Billie Eilish,pop,track,2024
8,Soltera,Shakira,pop,track,2024
9,El Principio de algo,La La Love You,pop,track,2023


In [17]:
#Información del DataFrame.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1138 entries, 0 to 1137
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Nombre_cancion  1138 non-null   object
 1   Nombre_artista  1138 non-null   object
 2   Genero          1138 non-null   object
 3   Tipo            1138 non-null   object
 4   Anio            1138 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 44.6+ KB


In [18]:
#Informar del número de columnas.
print(df.columns)

Index(['Nombre_cancion', 'Nombre_artista', 'Genero', 'Tipo', 'Anio'], dtype='object')


In [ ]:
#NO EJECUTAR!!!!!! Transformar el tipo de dato de la columna 'Anio' a int y eliminar los string.
df['Anio'] = pd.to_numeric(df['Anio'], errors='coerce')
# Elimina los datos con valores NaN en la columna 'Anio'
df = df.dropna(subset=['Anio'])
df['Anio'] = df['Anio'].astype(int)

#Explicación CHAT GPT (Celda anterior)

1. Convierte los valores a numéricos. Si hay strings como 'abc', se convierten a NaN
df['Anio'] = pd.to_numeric(df['Anio'], errors='coerce')

2. Elimina las filas donde 'Anio' es NaN (es decir, no eran números válidos)
df = df.dropna(subset=['Anio'])

3. Convierte la columna a entero
df['Anio'] = df['Anio'].astype(int)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1145 entries, 0 to 1144
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Nombre_cancion  1145 non-null   object
 1   Nombre_artista  1145 non-null   object
 2   Genero          1145 non-null   object
 3   Tipo            1145 non-null   object
 4   Anio            1145 non-null   int32 
dtypes: int32(1), object(4)
memory usage: 40.4+ KB


In [ ]:
#NO EJECUTAR!!!!! Cambiar el orden de las columnas para que coincidan con la base de datos SQL.
nuevo_orden = ['Nombre_artista', 'Tipo','Nombre_cancion', 'Genero', 'Anio']
df = df[nuevo_orden]

In [ ]:
# NO EJECUTAR!!!!!! Guardar el DataFrame en un archivo CSV sin duplicados
df.to_csv ('CSV datos_spotify_3_sin_duplicados.csv', index=False)

In [21]:
# Establecer la conexión
cnx = mysql.connector.connect(
    user='root', 
    password='AlumnaAdalab', 
    host='127.0.0.1'
)

print("Conexión establecida con éxito")

Conexión establecida con éxito


In [22]:
#Conectar a la base de datos
mycursor = cnx.cursor()
query = "USE music_stream2"
mycursor.execute(query)

In [23]:
# Insertar los datos en la tabla 'spotify' de la base de datos 'music_stream'.
#Si el archivo esta en la misma carpeta que el script, no es necesario especificar la ruta completa.
# Si el archivo CSV está en una ruta diferente, asegúrate de proporcionar la ruta completa. (r´).

df = pd.read_csv('CSV datos_spotify_3_sin_duplicados.csv', header=0, names=['Nombre_artista', 'Tipo','Nombre_cancion', 'Genero', 'Anio'])

values = df.values.tolist()

    # Consulta SQL para insertar varios valores en la tabla clientes
query_insert = '''
    INSERT INTO spotify (Nombre_artista, Tipo,Nombre_cancion, Genero, Anio)
    VALUES (%s, %s, %s, %s, %s)
    '''
    # Insertar los valores en la tabla clientes
mycursor.executemany(query_insert, values)
# Confirmar los cambios
cnx.commit()

In [18]:
cnx.close()